In [ ]:
from Algorithm import Gaussian_Process
from Algorithm import LA_Bayessian_Quadrature
from Algorithm import Mesh
from Algorithm import Kernels
import Target_Function as tf

import numpy as np


In [ ]:
opt_spy  = tf.Option('call', 0, 600, 0.1672, 0.15,  1, 100)   # SPDR S&P 500 ETF  (广义市场基准)
opt_aapl = tf.Option('call', 1, 200, 0.2613, 0.49,  1, 100)   # Apple
opt_tsla = tf.Option('call', 2, 350, 0.6398, 0.57,  1, 100)   # Tesla
opt_nvda = tf.Option('call', 3, 145, 0.3482, 0.24,  1, 100)   # NVIDIA
opt_amzn = tf.Option('call', 4, 210, 0.2902, 0.24,  1, 100)   # Amazon
opt_meta = tf.Option('call', 5, 700, 0.2924, 0.24,  1, 100)   # Meta
opt_msft = tf.Option('call', 6, 485, 0.1817, 0.15,  1, 100)   # Microsoft
opt_goog = tf.Option('call', 7, 165, 0.3226, 0.32,  1, 100)   # Google
opt_amd  = tf.Option('call', 8, 130, 0.4065, 0.32,  1, 100)   # AMD
opt_mstr = tf.Option('call', 9, 370, 0.5098, 0.49,  1, 100)   # MicroStrategy

def LABQ(D):
    options = [opt_spy, opt_aapl, opt_tsla, opt_nvda, opt_amzn,
               opt_meta, opt_msft, opt_goog, opt_amd, opt_mstr]
    portfolio = tf.OptionPortfolio(
        0.04,50,900,
        options[:D]
    )
    D = len(portfolio.options)
    func = portfolio.target_function
    xx = np.linspace(0., 1., 3)
    grids = np.meshgrid(*([xx] * D),)
    X = np.vstack([g.ravel() for g in grids]).T
    Y = np.array([func(x) for x in X])
    beta = np.concatenate([
    np.array([-0.5]),
    np.random.uniform(-2, 0, size=11 * D)
    ])
    mesh1 = Mesh.get_points_Sobol(X,_)
    GP = Gaussian_Process.GaussianProcess(Kernels.K_lin_mat2_10,beta,X,Y,mesh = mesh1)
    sobol = [8,7,6,5,4,3,3,2,2]
    a, b, time, uncertainty = LA_Bayessian_Quadrature.ABC_D(
        func,
        GP,
        Mesh.get_points_Sobol,
        n = min(30,max(49-6*D,3)),
        lambd = [30, 1],
        point_mesh = mesh1,
        sobol = sobol[D-2] #2^sobol points
    )
    return time,uncertainty

result = []
for D in range(2,11):
    """
    2D --> 31 Steps, Sobol: 2^7
    3D --> 31 Steps, Sobol: 2^7
    4D --> 26 Steps, Sobol: 2^7
    5D --> 20 Steps, Sobol: 2^7
    6D --> 14 Steps, Sobol: 2^6
    7D --> 8 Steps, Sobol: 2^5
    8D --> 4 Steps, Sobol: 2^4
    9D --> 4 Steps, Sobol: 2^3
    10D --> 4 Steps, Sobol: 2^2
    """
    time, uncertainty = LABQ(D)
    result.append([D,time,uncertainty])
print(result)
"""
note of the result:
Dimensions --> numbers of options
Sobol --> 2^sobol points every step
Steps --> how many cycle to run
Time --> time of each step
Result[mean, var] -->
To standardize the values of each point for calculation purposes.
The mean IS NOT final delta, it need to *Target_Function.portfolio.total

Total_time(at the end of each dimension test) --> total time of each dimension test
Uncertainty Rate --> sqrt(var) / mean * 100%

There's Two test result below, the second one runs for 10hrs, but 7D can't even go 1Step, but 6D is worked out.
Device: Macbook Pro with M3 Pro
"""

In [ ]:
# Dimensions: 2 , Sobol: 7 , Steps: 31
# Step 1 of 31, Time: 9.75, Result: [0.41411782 0.01301929]
# Step 2 of 31, Time: 9.34, Result: [0.45494964 0.00954901]
# Step 3 of 31, Time: 9.31, Result: [0.51606036 0.00661025]
# Step 4 of 31, Time: 9.61, Result: [0.57897944 0.00466921]
# Step 5 of 31, Time: 10.02, Result: [0.58858047 0.00208796]
# Step 6 of 31, Time: 11.08, Result: [0.59430979 0.00165366]
# Step 7 of 31, Time: 9.74, Result: [0.60016553 0.00133804]
# Step 8 of 31, Time: 9.58, Result: [0.587258   0.00110961]
# Step 9 of 31, Time: 9.77, Result: [0.58925585 0.00089059]
# Step 10 of 31, Time: 9.57, Result: [0.59035074 0.00072943]
# Step 11 of 31, Time: 9.59, Result: [0.5880381 0.000608 ]
# Step 12 of 31, Time: 9.39, Result: [5.83171795e-01 4.95100859e-04]
# Step 13 of 31, Time: 9.77, Result: [5.84418502e-01 4.17418824e-04]
# Step 14 of 31, Time: 9.65, Result: [5.85992759e-01 3.48372079e-04]
# Step 15 of 31, Time: 9.89, Result: [5.84731790e-01 2.95785634e-04]
# Step 16 of 31, Time: 9.96, Result: [5.85834490e-01 2.46337114e-04]
# Step 17 of 31, Time: 10.45, Result: [5.86259299e-01 2.08344747e-04]
# Step 18 of 31, Time: 9.79, Result: [5.87261131e-01 1.66935071e-04]
# Step 19 of 31, Time: 9.48, Result: [5.86534761e-01 1.33938880e-04]
# Step 20 of 31, Time: 9.56, Result: [5.87166249e-01 1.09643674e-04]
# Step 21 of 31, Time: 9.53, Result: [5.88311341e-01 9.29456880e-05]
# Step 22 of 31, Time: 9.52, Result: [5.86876122e-01 8.36972955e-05]
# Step 23 of 31, Time: 9.57, Result: [5.87854778e-01 7.28428271e-05]
# Step 24 of 31, Time: 9.65, Result: [5.91016518e-01 6.69002242e-05]
# Step 25 of 31, Time: 9.77, Result: [5.91176544e-01 5.99145639e-05]
# Step 26 of 31, Time: 9.70, Result: [5.91911141e-01 5.35056273e-05]
# Step 27 of 31, Time: 9.81, Result: [5.91573186e-01 4.67846205e-05]
# Step 28 of 31, Time: 9.90, Result: [5.91487972e-01 4.13768482e-05]
# Step 29 of 31, Time: 10.44, Result: [5.91376329e-01 3.62740925e-05]
# Step 30 of 31, Time: 9.87, Result: [5.91421688e-01 3.19875849e-05]
# Step 31 of 31, Time: 9.89, Result: [5.91566864e-01 2.81602974e-05]
# 2Dimensions Total_Time: 294.06
# Uncertainty Rate: 0.90%
# Dimensions: 3 , Sobol: 7 , Steps: 31
# Step 1 of 31, Time: 17.37, Result: [0.42569517 0.00789137]
# Step 2 of 31, Time: 16.15, Result: [0.45286466 0.00675989]
# Step 3 of 31, Time: 17.01, Result: [0.46655446 0.0055546 ]
# Step 4 of 31, Time: 16.12, Result: [0.49452805 0.00464921]
# Step 5 of 31, Time: 15.78, Result: [0.52988136 0.00373022]
# Step 6 of 31, Time: 16.47, Result: [0.54993851 0.00327953]
# Step 7 of 31, Time: 16.57, Result: [0.55771529 0.00274972]
# Step 8 of 31, Time: 17.55, Result: [0.56631522 0.00229143]
# Step 9 of 31, Time: 16.52, Result: [0.58006107 0.00183098]
# Step 10 of 31, Time: 16.49, Result: [0.58857669 0.00163003]
# Step 11 of 31, Time: 15.45, Result: [0.59436831 0.00145102]
# Step 12 of 31, Time: 16.65, Result: [0.5967775 0.0012912]
# Step 13 of 31, Time: 18.00, Result: [0.59796065 0.00118216]
# Step 14 of 31, Time: 17.14, Result: [0.59805001 0.00109376]
# Step 15 of 31, Time: 16.51, Result: [0.59752396 0.00100974]
# Step 16 of 31, Time: 16.46, Result: [0.59874263 0.00093967]
# Step 17 of 31, Time: 17.35, Result: [0.59507966 0.00086841]
# Step 18 of 31, Time: 18.27, Result: [0.59809547 0.00081038]
# Step 19 of 31, Time: 17.76, Result: [0.5998022  0.00075033]
# Step 20 of 31, Time: 18.57, Result: [0.60108048 0.00070234]
# Step 21 of 31, Time: 17.52, Result: [0.59881939 0.00065221]
# Step 22 of 31, Time: 18.32, Result: [0.59688832 0.00060403]
# Step 23 of 31, Time: 18.41, Result: [5.99866045e-01 5.64411961e-04]
# Step 24 of 31, Time: 18.90, Result: [5.99843838e-01 5.23145008e-04]
# Step 25 of 31, Time: 19.34, Result: [5.98690488e-01 4.87697405e-04]
# Step 26 of 31, Time: 17.72, Result: [5.99546179e-01 4.57424207e-04]
# Step 27 of 31, Time: 17.30, Result: [5.99585442e-01 4.27138403e-04]
# Step 28 of 31, Time: 19.42, Result: [6.00861611e-01 4.02140888e-04]
# Step 29 of 31, Time: 21.05, Result: [6.01361107e-01 3.76413558e-04]
# Step 30 of 31, Time: 20.88, Result: [6.02111709e-01 3.50500574e-04]
# Step 31 of 31, Time: 18.74, Result: [6.04194211e-01 3.27605089e-04]
# 3Dimensions Total_Time: 528.50
# Uncertainty Rate: 3.00%
# Dimensions: 4 , Sobol: 7 , Steps: 26
# Step 1 of 26, Time: 37.17, Result: [0.42845772 0.00414153]
# Step 2 of 26, Time: 43.26, Result: [0.45069437 0.0038568 ]
# Step 3 of 26, Time: 22.87, Result: [0.46644515 0.00354082]
# Step 4 of 26, Time: 24.13, Result: [0.48310719 0.00327039]
# Step 5 of 26, Time: 30.66, Result: [0.50321386 0.00300663]
# Step 6 of 26, Time: 27.53, Result: [0.51198398 0.00272108]
# Step 7 of 26, Time: 28.24, Result: [0.52546976 0.00250688]
# Step 8 of 26, Time: 31.20, Result: [0.53540217 0.00232895]
# Step 9 of 26, Time: 29.64, Result: [0.54611095 0.00217373]
# Step 10 of 26, Time: 30.34, Result: [0.55492856 0.00200781]
# Step 11 of 26, Time: 29.61, Result: [0.559103   0.00189451]
# Step 12 of 26, Time: 37.38, Result: [0.56713871 0.00178047]
# Step 13 of 26, Time: 25.67, Result: [0.57502028 0.00168688]
# Step 14 of 26, Time: 38.05, Result: [0.57812764 0.00159403]
# Step 15 of 26, Time: 30.37, Result: [0.5828605  0.00149044]
# Step 16 of 26, Time: 26.87, Result: [0.58675167 0.0014075 ]
# Step 17 of 26, Time: 28.45, Result: [0.59151532 0.00133667]
# Step 18 of 26, Time: 34.21, Result: [0.59903247 0.00127145]
# Step 19 of 26, Time: 29.60, Result: [0.60238518 0.00120742]
# Step 20 of 26, Time: 28.96, Result: [0.60586776 0.00115613]
# Step 21 of 26, Time: 24.40, Result: [0.60900055 0.00108987]
# Step 22 of 26, Time: 31.86, Result: [0.61148593 0.0010455 ]
# Step 23 of 26, Time: 25.44, Result: [0.61416906 0.00100355]
# Step 24 of 26, Time: 22.97, Result: [0.61612385 0.00096427]
# Step 25 of 26, Time: 28.02, Result: [0.61915344 0.00092507]
# Step 26 of 26, Time: 25.82, Result: [0.62276933 0.00088665]
# 4Dimensions Total_Time: 748.54
# Uncertainty Rate: 4.78%
# Dimensions: 5 , Sobol: 7 , Steps: 20
# Step 1 of 20, Time: 226.98, Result: [0.41371399 0.00192233]
# Step 2 of 20, Time: 109.73, Result: [0.42670782 0.00184439]
# Step 3 of 20, Time: 75.01, Result: [0.43851689 0.00176368]
# Step 4 of 20, Time: 117.30, Result: [0.44667755 0.00167964]
# Step 5 of 20, Time: 130.35, Result: [0.45465709 0.00159984]
# Step 6 of 20, Time: 49.27, Result: [0.46078034 0.00154155]
# Step 7 of 20, Time: 50.95, Result: [0.4672851  0.00147933]
# Step 8 of 20, Time: 71.59, Result: [0.47428623 0.00142044]
# Step 9 of 20, Time: 65.46, Result: [0.48165606 0.00136584]
# Step 10 of 20, Time: 72.34, Result: [0.4874515 0.0013155]
# Step 11 of 20, Time: 73.86, Result: [0.49556114 0.00126841]
# Step 12 of 20, Time: 69.25, Result: [0.50376894 0.00123283]
# Step 13 of 20, Time: 63.37, Result: [0.50699893 0.00119882]
# Step 14 of 20, Time: 67.47, Result: [0.51201211 0.00116447]
# Step 15 of 20, Time: 62.06, Result: [0.51787333 0.0011268 ]
# Step 16 of 20, Time: 81.48, Result: [0.52175924 0.00109211]
# Step 17 of 20, Time: 69.84, Result: [0.52537274 0.00106476]
# Step 18 of 20, Time: 57.40, Result: [0.53081603 0.0010331 ]
# Step 19 of 20, Time: 75.26, Result: [0.5344749  0.00100205]
# Step 20 of 20, Time: 73.91, Result: [0.54094532 0.0009656 ]
# 5Dimensions Total_Time: 1590.69
# Uncertainty Rate: 5.74%
# Dimensions: 6 , Sobol: 6 , Steps: 14
# Step 1 of 14, Time: 2279.42, Result: [0.34033529 0.00071755]

In [ ]:
# Dimensions: 2 , Sobol: 8 , Steps: 31
# Step 1 of 31, Time: 9.71, Result: [0.41411796 0.0130193 ]
# Step 2 of 31, Time: 12.91, Result: [0.4535456  0.00952509]
# Step 3 of 31, Time: 12.82, Result: [0.51256884 0.00661382]
# Step 4 of 31, Time: 13.29, Result: [0.56994492 0.00453967]
# Step 5 of 31, Time: 12.98, Result: [0.58006665 0.0020103 ]
# Step 6 of 31, Time: 12.94, Result: [0.58499509 0.00158595]
# Step 7 of 31, Time: 12.95, Result: [0.57525468 0.00126772]
# Step 8 of 31, Time: 12.92, Result: [0.57758361 0.00102346]
# Step 9 of 31, Time: 13.02, Result: [0.57944671 0.00082648]
# Step 10 of 31, Time: 13.03, Result: [0.58305578 0.00067366]
# Step 11 of 31, Time: 12.98, Result: [5.85019692e-01 5.61634094e-04]
# Step 12 of 31, Time: 13.08, Result: [5.89408409e-01 4.80477388e-04]
# Step 13 of 31, Time: 13.19, Result: [5.90964715e-01 3.91416295e-04]
# Step 14 of 31, Time: 12.90, Result: [5.88575637e-01 3.22604613e-04]
# Step 15 of 31, Time: 13.02, Result: [5.88819549e-01 2.63934611e-04]
# Step 16 of 31, Time: 13.05, Result: [5.89256584e-01 2.22509233e-04]
# Step 17 of 31, Time: 13.11, Result: [5.92089940e-01 1.75544925e-04]
# Step 18 of 31, Time: 13.00, Result: [5.93136863e-01 1.47295218e-04]
# Step 19 of 31, Time: 13.76, Result: [5.90863845e-01 1.20203626e-04]
# Step 20 of 31, Time: 12.87, Result: [5.88194095e-01 1.10171582e-04]
# Step 21 of 31, Time: 13.06, Result: [5.89007271e-01 9.65862806e-05]
# Step 22 of 31, Time: 12.97, Result: [5.88504975e-01 8.32026724e-05]
# Step 23 of 31, Time: 13.04, Result: [5.89719165e-01 7.46705519e-05]
# Step 24 of 31, Time: 13.05, Result: [5.90195436e-01 6.22326644e-05]
# Step 25 of 31, Time: 13.48, Result: [5.89998224e-01 5.46788867e-05]
# Step 26 of 31, Time: 13.63, Result: [5.89809692e-01 4.57307687e-05]
# Step 27 of 31, Time: 13.98, Result: [5.91050732e-01 4.08024500e-05]
# Step 28 of 31, Time: 14.13, Result: [5.91299374e-01 3.50075249e-05]
# Step 29 of 31, Time: 13.29, Result: [5.90864411e-01 3.00089380e-05]
# Step 30 of 31, Time: 13.36, Result: [5.92877185e-01 2.56500809e-05]
# Step 31 of 31, Time: 13.30, Result: [5.92484479e-01 2.23667781e-05]
# 2Dimensions Total_Time: 393.51
# Uncertainty Rate: 0.80%
# Dimensions: 3 , Sobol: 7 , Steps: 31
# Step 1 of 31, Time: 16.62, Result: [0.4256954  0.00789139]
# Step 2 of 31, Time: 16.67, Result: [0.46151414 0.00690985]
# Step 3 of 31, Time: 15.85, Result: [0.49855739 0.00590234]
# Step 4 of 31, Time: 17.42, Result: [0.52475336 0.00483766]
# Step 5 of 31, Time: 16.42, Result: [0.53684125 0.00392297]
# Step 6 of 31, Time: 16.47, Result: [0.55163715 0.00331962]
# Step 7 of 31, Time: 17.97, Result: [0.5565141  0.00278898]
# Step 8 of 31, Time: 17.75, Result: [0.56198435 0.00237773]
# Step 9 of 31, Time: 16.78, Result: [0.58076115 0.00199421]
# Step 10 of 31, Time: 16.04, Result: [0.58151745 0.00176293]
# Step 11 of 31, Time: 16.25, Result: [0.58075642 0.00154983]
# Step 12 of 31, Time: 17.25, Result: [0.5851335  0.00139685]
# Step 13 of 31, Time: 16.29, Result: [0.58314456 0.00126386]
# Step 14 of 31, Time: 16.42, Result: [0.58182832 0.0011454 ]
# Step 15 of 31, Time: 16.55, Result: [0.58060048 0.0010358 ]
# Step 16 of 31, Time: 17.34, Result: [0.58486521 0.00095307]
# Step 17 of 31, Time: 16.58, Result: [0.58637917 0.00087763]
# Step 18 of 31, Time: 18.34, Result: [0.58790272 0.00080202]
# Step 19 of 31, Time: 18.48, Result: [0.58890293 0.00073944]
# Step 20 of 31, Time: 17.71, Result: [0.59114272 0.00068129]
# Step 21 of 31, Time: 17.83, Result: [0.59330015 0.00062787]
# Step 22 of 31, Time: 18.70, Result: [5.95237547e-01 5.76540764e-04]
# Step 23 of 31, Time: 17.16, Result: [5.98169425e-01 5.42202249e-04]
# Step 24 of 31, Time: 17.68, Result: [5.99119004e-01 5.09429262e-04]
# Step 25 of 31, Time: 17.82, Result: [6.01022124e-01 4.81989367e-04]
# Step 26 of 31, Time: 17.98, Result: [6.01257182e-01 4.55077998e-04]
# Step 27 of 31, Time: 18.88, Result: [6.01472885e-01 4.30481174e-04]
# Step 28 of 31, Time: 18.62, Result: [6.01409130e-01 4.02455568e-04]
# Step 29 of 31, Time: 18.76, Result: [6.01000766e-01 3.77917383e-04]
# Step 30 of 31, Time: 17.62, Result: [6.00454070e-01 3.54019239e-04]
# Step 31 of 31, Time: 17.68, Result: [6.00617422e-01 3.33483021e-04]
# 3Dimensions Total_Time: 521.73
# Uncertainty Rate: 3.04%
# Dimensions: 4 , Sobol: 6 , Steps: 26
# Step 1 of 26, Time: 30.97, Result: [0.4284554  0.00414149]
# Step 2 of 26, Time: 19.78, Result: [0.44219677 0.00382028]
# Step 3 of 26, Time: 19.29, Result: [0.45929062 0.0035488 ]
# Step 4 of 26, Time: 39.30, Result: [0.47947006 0.00333532]
# Step 5 of 26, Time: 31.41, Result: [0.49894708 0.00304566]
# Step 6 of 26, Time: 20.62, Result: [0.514343   0.00282099]
# Step 7 of 26, Time: 20.70, Result: [0.5281165  0.00259988]
# Step 8 of 26, Time: 23.41, Result: [0.53465159 0.0023713 ]
# Step 9 of 26, Time: 32.42, Result: [0.54382458 0.00221921]
# Step 10 of 26, Time: 22.05, Result: [0.55042168 0.00209911]
# Step 11 of 26, Time: 27.48, Result: [0.56067877 0.00198195]
# Step 12 of 26, Time: 20.42, Result: [0.56757494 0.00184683]
# Step 13 of 26, Time: 36.69, Result: [0.57045832 0.00173017]
# Step 14 of 26, Time: 37.01, Result: [0.57806093 0.00162416]
# Step 15 of 26, Time: 25.09, Result: [0.58154794 0.00152849]
# Step 16 of 26, Time: 38.91, Result: [0.59034127 0.00143954]
# Step 17 of 26, Time: 19.94, Result: [0.59761021 0.00136662]
# Step 18 of 26, Time: 29.30, Result: [0.59953706 0.001316  ]
# Step 19 of 26, Time: 26.23, Result: [0.60303241 0.00124556]
# Step 20 of 26, Time: 26.31, Result: [0.60505685 0.00118223]
# Step 21 of 26, Time: 30.49, Result: [0.6107717  0.00112583]
# Step 22 of 26, Time: 25.54, Result: [0.61428242 0.00107975]
# Step 23 of 26, Time: 27.01, Result: [0.61672979 0.00102743]
# Step 24 of 26, Time: 29.61, Result: [0.61757448 0.00098863]
# Step 25 of 26, Time: 24.72, Result: [0.62063607 0.00094942]
# Step 26 of 26, Time: 27.60, Result: [0.62073347 0.0009158 ]
# 4Dimensions Total_Time: 685.54
# Uncertainty Rate: 4.88%
# Dimensions: 5 , Sobol: 5 , Steps: 20
# Step 1 of 20, Time: 162.81, Result: [0.41372361 0.00192354]
# Step 2 of 20, Time: 69.91, Result: [0.42726909 0.00184664]
# Step 3 of 20, Time: 145.83, Result: [0.43969126 0.00176347]
# Step 4 of 20, Time: 108.93, Result: [0.44778468 0.0016889 ]
# Step 5 of 20, Time: 57.83, Result: [0.45415328 0.00162173]
# Step 6 of 20, Time: 48.54, Result: [0.46142573 0.00156054]
# Step 7 of 20, Time: 53.73, Result: [0.46906718 0.00149383]
# Step 8 of 20, Time: 60.54, Result: [0.47664562 0.00143922]
# Step 9 of 20, Time: 53.96, Result: [0.4866726  0.00137842]
# Step 10 of 20, Time: 57.96, Result: [0.49379571 0.00133385]
# Step 11 of 20, Time: 53.05, Result: [0.4993693  0.00128368]
# Step 12 of 20, Time: 151.88, Result: [0.50275756 0.00124519]
# Step 13 of 20, Time: 60.59, Result: [0.50927722 0.00120706]
# Step 14 of 20, Time: 52.52, Result: [0.51581964 0.00116883]
# Step 15 of 20, Time: 51.92, Result: [0.52175147 0.00113302]
# Step 16 of 20, Time: 59.73, Result: [0.52787608 0.00110839]
# Step 17 of 20, Time: 51.33, Result: [0.52991166 0.00108474]
# Step 18 of 20, Time: 46.38, Result: [0.53302241 0.0010639 ]
# Step 19 of 20, Time: 66.16, Result: [0.53774428 0.00103972]
# Step 20 of 20, Time: 115.55, Result: [0.54106896 0.0010145 ]
# 5Dimensions Total_Time: 1414.08
# Uncertainty Rate: 5.89%
# Dimensions: 6 , Sobol: 4 , Steps: 14
# Step 1 of 14, Time: 2674.16, Result: [0.34032899 0.00071745]
# Step 2 of 14, Time: 463.81, Result: [0.34483864 0.00069861]
# Step 3 of 14, Time: 695.51, Result: [0.34941228 0.00068586]
# Step 4 of 14, Time: 753.30, Result: [0.35461333 0.00066797]
# Step 5 of 14, Time: 847.26, Result: [0.35757548 0.00065342]
# Step 6 of 14, Time: 1134.76, Result: [0.36128074 0.00064461]
# Step 7 of 14, Time: 366.37, Result: [0.36554043 0.00063516]
# Step 8 of 14, Time: 638.17, Result: [0.36833598 0.00062306]
# Step 9 of 14, Time: 1088.84, Result: [0.37253498 0.000614  ]
# Step 10 of 14, Time: 879.00, Result: [0.3758705 0.0006067]
# Step 11 of 14, Time: 566.62, Result: [0.37969078 0.0005975 ]
# Step 12 of 14, Time: 1510.66, Result: [0.38322295 0.00058819]
# Step 13 of 14, Time: 4701.50, Result: [0.38579849 0.00057601]
# Step 14 of 14, Time: 4638.48, Result: [0.38992604 0.00056808]
# 6Dimensions Total_Time: 16320.41
# Uncertainty Rate: 6.11%